<a href="https://colab.research.google.com/github/Sai-sakunthala/hybrid-quantum-classical-algorithm-implementation-for-entropy-calculation/blob/main/bell_state.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pennylane torch --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93

In [ ]:
import pennylane as qml
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Set up device
n_qubits = 2
n_shots = 5000
dev = qml.device("default.qubit", wires=n_qubits, shots=n_shots)

@qml.qnode(dev)
def bell_sampler():
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    return qml.sample(qml.PauliZ(0)), qml.sample(qml.PauliZ(1))

def get_bell_samples():
    z0, z1 = bell_sampler()
    # Convert PauliZ outputs {-1, +1} → {1, 0}
    z0_bin = ((1 - z0) / 2).astype(np.float32)
    z1_bin = ((1 - z1) / 2).astype(np.float32)

    # Stack into shape (n_samples, 2)
    bitstrings = np.stack([z0_bin, z1_bin], axis=-1)
    return torch.tensor(bitstrings, dtype=torch.float32)

def get_uniform_samples(n_samples, n_bits):
    return torch.randint(0, 2, (n_samples, n_bits)).float()

In [ ]:
class EntropyNet(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.model(x)

def parametric_qnee_cost(model, real_samples, uniform_samples):
    d = 2 ** real_samples.shape[1]
    term1 = torch.mean(model(real_samples))
    term2 = torch.mean(torch.exp(model(uniform_samples)))
    cost = -term1 + term2
    rel_entropy = 1 - cost
    entropy = np.log(d) - 1 - cost
    return cost, entropy, rel_entropy

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EntropyNet(input_dim=2).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.5)


for epoch in range(1001):
    real_samples = get_bell_samples().to(device)
    uniform_samples = get_uniform_samples(len(real_samples), 2).to(device)

    loss, entropy, rel_entropy = parametric_qnee_cost(model, real_samples, uniform_samples)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f"Epoch {epoch}: Estimated von Neumann Entropy = {entropy.item():.4f}")
        print(f"Epoch {epoch}: Estimated relative entropy between bell state and maximally mixed state = {rel_entropy.item():.4f}")
        print('')

Epoch 0: Estimated von Neumann Entropy = -0.6454
Epoch 0: Estimated relative entropy between bell state and maximally mixed state = -0.0317

Epoch 100: Estimated von Neumann Entropy = -0.4844
Epoch 100: Estimated relative entropy between bell state and maximally mixed state = 0.1293

Epoch 200: Estimated von Neumann Entropy = -0.3503
Epoch 200: Estimated relative entropy between bell state and maximally mixed state = 0.2634

Epoch 300: Estimated von Neumann Entropy = -0.2293
Epoch 300: Estimated relative entropy between bell state and maximally mixed state = 0.3844

Epoch 400: Estimated von Neumann Entropy = -0.1121
Epoch 400: Estimated relative entropy between bell state and maximally mixed state = 0.5016

Epoch 500: Estimated von Neumann Entropy = -0.0347
Epoch 500: Estimated relative entropy between bell state and maximally mixed state = 0.5790

Epoch 600: Estimated von Neumann Entropy = -0.0046
Epoch 600: Estimated relative entropy between bell state and maximally mixed state = 0.6